In [3]:
import pandas as pd
import numpy as np
import random
import random
import string
import json
import csv

In [25]:
## Importazione dati e definizione costanti

df_HIT_reviews = pd.read_csv('df_HIT_reviews.csv')
df_HIT_reviews

df_HITs = pd.read_csv('HITs.csv')
df_HITs

## nr di righe nel df
UNITS= df_HIT_reviews.shape[0] 

## path per creare il file hits.json all'interno di Crowd_Frame
CROWDFRAMEPATH= 'Crowd_Frame riscaricato/data/build/task/'

In [26]:
## Classe Task

## Un singolo Task è generato  in base a una riga del data-frame contenente gli Hits (df_HITs)
## unit_id: identificativo della task: stringa
## token.input: token casuale: stringa
## token.output: token casuale: stringa
## documents: array contente singoli "documento" rappresentati una HIT
##
##
## struttura del singolo documento
## 
## documento = {
##             "id": asin,
##             "review_time": , 
##             "product_title": , 
##             "product_brand": ,
##             "review_text": 
##         }
## 
class Task:
    
    def __init__(self, i, my_df):
        self.index = i
        self.unit_id = None #self.set_unit_id(my_df, i)
        self.token_input = None
        self.token_output = None
        self.documents = None
        
        self.set_unit_id(my_df, i)
        self.token_input = self.set_token()
        self.token_output = self.set_token()
        self.set_documents(df=df_HITs)
        
    
    ## ricerca unit_id    
    ## df: df dove si trova unit_id
    ## row: riga dove si trova unit_id
    def set_unit_id(self, df, row):
        unit_id_value = df.at[row, 'HIT']
        self.unit_id= unit_id_value
        
    
    ## genera un token casuale composto da 11 caratteri maiuscoli
    def set_token(self):
        lunghezza = 11
        caratteri_maiuscoli = string.ascii_uppercase
        stringa_generata = ''.join(random.choice(caratteri_maiuscoli) for _ in range(lunghezza))
        return stringa_generata
    
    
    ## setta i documenti presenti all'interno dell'array documents
    def set_documents(self, df):
        
        documents=[]
        
        asins = df.iloc[self.index, 1:]
        
        for asin in asins:
            document = self.__document(df=df_HIT_reviews, asin=asin)
            documents.append(document)
        
        self.documents= documents
    
    ## Ritorna la lunghezza dell'array documents
    def document_length(self):
        return len(self.documents)
    
    ## Private
    ## estrae i dati dal df contenente le reviews
    def __document(self, df, asin):

        documento = {
            "id": asin,
            "review_time":df.loc[df['product_asin'] == asin, 'review_time'].values[0], 
            "product_title": df.loc[df['product_asin'] == asin, 'product_title'].values[0], 
            "product_brand": df.loc[df['product_asin'] == asin, 'product_brand'].values[0],
            "review_text": df.loc[df['product_asin'] == asin, 'review_text'].values[0]
        }
        return documento

In [27]:
## Rappresenta l'array che contiene le parti fondamentali di una Task
def create_unit(task):
        
    data = {
        "unit_id": task.unit_id,
        "token_input": task.token_input,
        "token_output": task.token_output,
        "documents_number": task.document_length(),
        "documents": task.documents 
    }

    return data

    

In [28]:
## Array che contiene tutte le HITS
HITS = [] 

for i in range(0,UNITS):
    hit= Task(i, my_df=df_HITs)
    data = create_unit(hit)
    
    HITS.append(data)

## Crea il file dentro Crowd_Frame    
with open(CROWDFRAMEPATH + 'hits.json', 'w') as json_file:
    json.dump(HITS, json_file)

## Crea il file di controllo   
with open('hits.json', 'w') as json_file:
    json.dump(HITS, json_file)

In [27]:
def extract_tokens_from_json():
    
    with open('hits.json', 'r') as file:
                # Load JSON data from the file
                data = json.load(file)
    
                # Extract all tokens
                tokens_list = []
                for item in data:
                    token_input = item.get("token_input")
                    token_output = item.get("token_output")
                    tokens_list.append(token_input)
                    tokens_list.append(token_output)
                
    return tokens_list

def save_list_to_csv(data_list):
        
    with open('tokens.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['tokens'] + [''] * (len(data_list) - 1))  # Scritta 'tokens' nella prima colonna
        for _ in range(15):
            csv_writer.writerow(data_list)


In [28]:
data = extract_tokens_from_json()
save_list_to_csv(data)